*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling


import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

from xgboost import XGBRegressor

#xgboost triggers a deprecation warning which we ignore here 
import warnings
warnings.filterwarnings(action='ignore')

%matplotlib inline

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
df = pd.read_csv('insurance.csv')
df.tail()

In [ ]:
# take a first look at the data
df.head()

In [ ]:
# inspect column distributions before exploratory data analysis (EDA)

df.describe(include="all")

In [ ]:
# we have no missing values and 4 numerical features (possibly children might not be one, we have to visualize that column later) 
df.info()

In [ ]:
# map gender, smoker
df['smoker'] = df['smoker'].map({"no": 0, 'yes': 1})
df = pd.get_dummies(df, prefix='', prefix_sep='')

print(df.head())

In [ ]:
# split data
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

In [ ]:
# remove expenses
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
# normalize data
normalizer = layers.experimental.preprocessing.Normalization()
normalizer.adapt(np.array(train_dataset))

In [ ]:
# linear regression model
model = keras.Sequential([
      normalizer,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss=tf.losses.MeanAbsoluteError(),
    metrics=[tf.metrics.MeanAbsoluteError(), tf.metrics.MeanSquaredError()])

model.summary()

history = model.fit(x=train_dataset, y=train_labels,
                    epochs=300,
                    verbose=0,
                    validation_split = 0.2)  # Calculate validation results on 20% of the training data

In [ ]:
# plot loss
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
#plot_loss(history)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
#loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

#print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

#if mae < 3500:
#  print("You passed the challenge. Great job!")
#else:
#  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.

#test_predictions = model.predict(test_dataset).flatten()

#a = plt.axes(aspect='equal')
#plt.scatter(test_labels, test_predictions)
#plt.xlabel('True values (expenses)')
#plt.ylabel('Predictions (expenses)')
#lims = [0, 50000]
#plt.xlim(lims)
#plt.ylim(lims)
#_ = plt.plot(lims,lims)


# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
plt.show()